In [1]:
# This changes the current directory to the base saga directory - make sure to run this first!
# This is necessary to be able to import the py files and use the right directories,
# while keeping all the notebooks in their own directory.
import os
import sys

if 'saga_base_dir' not in locals():
    saga_base_dir = os.path.abspath('..')
if saga_base_dir not in sys.path:
    os.chdir(saga_base_dir)

In [2]:
import urllib
import requests
import webbrowser
import datetime

import numpy as np

#astropy >= v1.1 needed
from astropy import units as u
from astropy import table 
from astropy.coordinates import SkyCoord

%matplotlib inline
from matplotlib import pyplot as plt

import targeting #from SAGA

The actual catalogs were downloaded using the `download_host_sqlfile.py` file from https://github.com/saga-survey/marla to the data directory below

In [3]:
data_dir = '../local_data/'

run this to *look at* the rem list in the browser

In [4]:
webbrowser.open(targeting._DEFAULT_TREM_URL.replace('/export?format=csv&', '#'))

True

Now save today's version

In [6]:
fnremlist = 'TargetRemove{}.csv'.format(datetime.datetime.now().strftime('%b%d_%Y'))

if not os.path.exists(fnremlist):
    urllib.request.urlretrieve(targeting._DEFAULT_TREM_URL, fnremlist)
fnremlist

'TargetRemoveDec16_2015.csv'

Load the remove list as an Astropy table, and add a "row_index" to make it easier to look at in the notebook

In [7]:
remlist = table.Table.read(fnremlist, format='csv',data_start=3, header_start=1)
remlist.remove_column('col6') # google made a blank column for some reason?

remlist.add_column(remlist.ColumnClass(name='row_index', data=np.arange(len(remlist))), index=0)
remlist = remlist[~remlist['SDSS ID'].mask]

remlist.show_in_notebook(display_length=10)

row_index,Host Name,NSA,SDSS ID,Targ_RA,Targ_Dec,comment
0,Hamlet,166035,1237662498391261226,228.38953,41.95437,--
1,Hamlet,166035,1237662300829384725,228.778293,42.209718,--
2,Hamlet,166035,1237662300829253660,228.373589,42.32577,--
3,Hamlet,166035,1237662300829384730,228.75958,42.217655,--
4,Hamlet,166035,1237662300829253654,228.368821,42.335368,--
5,Hamlet,166035,1237662300829253651,228.374792,42.330362,--
6,Hamlet,166035,1237662300829384720,228.763359,42.215231,--
7,Hamlet,166035,1237662300829189130,228.234658,42.295864,--
8,Hamlet,166035,1237662498928197694,229.038466,42.321874,--
9,Hamlet,166035,1237658205598712218,228.247022,41.844161,--


Load the catalogs of all NSA objects in the remove list and extract the flags for any of the targets that are actually in the remove list.  Also get flag-related statistics for *everything*

In [8]:
#for remlist comparison
tojoin = {'OBJID': [], 'FLAGS': []}

# for "all" stats
allcount = 0
all_flag_counts = {(i+1):0 for i in range(64)}
flag_masks = {(i+1):np.array(1 << i) for i in range(64)}

# have to loop over NSAIDs to efficiently load the catalogs one at a time
remlist_ids = remlist['SDSS ID']
unsaids = np.unique(remlist['NSA'])
for i, nsanum in enumerate(unsaids):
    print('On NSA#', nsanum, 'which is', i+1, 'of', len(unsaids))
    sys.stdout.flush()
    
    fn = os.path.join(data_dir, 'sql_nsa{}.fits.gz'.format(nsanum))
    try:
        tab = table.Table.read(fn)

        # find just the remove list objects' prpoerties
        tab_matched = tab[np.in1d(tab['OBJID'], remlist_ids)]
        for key, lst in tojoin.items():
            lst.extend(tab_matched[key])
            
        #extract FLAGS statstics for *all* objects
        for i, mask in flag_masks.items():
            all_flag_counts[i] += np.sum((tab['FLAGS'].astype('uint64')&mask)!=0)
        allcount += len(tab)
        
    except FileNotFoundError:
        print('Did not find', fn, 'so skipping it')

On NSA# 32 which is 1 of 28
On NSA# 13927 which is 2 of 28
On NSA# 33446 which is 3 of 28
On NSA# 61945 which is 4 of 28
On NSA# 61954 which is 5 of 28
Did not find ../local_data/sql_nsa61954.fits.gz so skipping it
On NSA# 85746 which is 6 of 28
On NSA# 126115 which is 7 of 28
On NSA# 129237 which is 8 of 28
On NSA# 129387 which is 9 of 28
On NSA# 130625 which is 10 of 28
On NSA# 131531 which is 11 of 28
On NSA# 132339 which is 12 of 28
On NSA# 140594 which is 13 of 28
On NSA# 145729 which is 14 of 28
On NSA# 148734 which is 15 of 28
On NSA# 149781 which is 16 of 28
On NSA# 149977 which is 17 of 28
On NSA# 150238 which is 18 of 28
On NSA# 150307 which is 19 of 28
On NSA# 150340 which is 20 of 28
On NSA# 150578 which is 21 of 28
On NSA# 150877 which is 22 of 28
Did not find ../local_data/sql_nsa150877.fits.gz so skipping it
On NSA# 150887 which is 23 of 28
On NSA# 153017 which is 24 of 28
On NSA# 165536 which is 25 of 28
On NSA# 166035 which is 26 of 28
On NSA# 166313 which is 27 of 28


Now combine the remove list table with the set of FLAGS we got above

In [9]:
if 'OBJID' in tojoin:
    tojoin['SDSS ID'] = tojoin.pop('OBJID') # rename for join
remwflags = table.join(remlist, table.Table(tojoin), join_type='left')
remwflags.show_in_notebook(display_length=10)

row_index,Host Name,NSA,SDSS ID,Targ_RA,Targ_Dec,comment,FLAGS
187,--,165536,123764872124891147,221.57195,-0.19887408,--,--
159,--,145729,1237648702984553095,224.6328147,-1.079987716,part of main galaxy,68987912960
181,--,165536,1237648703519850685,221.0636814,-0.6259872282,star spike,105622104838420
178,--,165536,1237648703519981589,221.3734899,-0.6247009016,star spike,105587745100124
179,--,165536,1237648703519981596,221.3687812,-0.6257746866,star spike,105656498131036
180,--,165536,1237648703519981597,221.3682296,-0.6248161704,star spike,105622104703324
165,--,145729,1237648703521554658,224.9393172,-0.6272479811,second part of merging galaxy,105555532321040
177,--,165536,1237648704056787393,221.2368418,-0.3005513682,second part of merger,35253360132112
175,--,165536,1237648704057050408,221.8626044,-0.3057260505,star spike,439804919677184
176,--,165536,1237648704057050409,221.8638732,-0.3042778795,star spike,281543964754176


Convert FLAGS value to how many objects have specific bits set 

Note that "bit #1" is the 0th bit (in python indexing)

In [10]:
flags = remwflags['FLAGS']
flags = flags[~flags.mask].view(np.ndarray).astype('uint64')

bitcounts = {}
for i in range(64):
    bitmask = np.array(1 << i, dtype='uint64')
    bitcounts[i+1] = np.sum((flags & bitmask) != 0)
    
bitcounttable = table.Table(data=[list(bitcounts.values()), list(bitcounts.keys())], names=['count', 'bit'])

Now get the description of each bit from SDSS SQL and combine that with the Counts from above

In [22]:
# this SQL query gets the meaning of the flags
qry = urllib.parse.urlencode({'cmd': 'SELECT * FROM PhotoFlags', 'format': 'csv'})

url = 'http://skyserver.sdss.org/dr12/en/tools/search/x_sql.aspx?' + qry
csv = urllib.request.urlopen(url).read()

In [26]:
names = []
descrs = []
bits = []
# now we hand-parse it to get the bit field right
rows = csv.split(b'\n')
for row in rows[2:]:  #skip the "Table1" and the header
    if row.strip() == b'':
        continue
        
    spl = row.split(b',')
    name, val = spl[:2]
    descr = b','.join(spl[2:])
    
    names.append(name)
    descrs.append(descr)
    
    intval = int(val, 0)
    for i in range(65):
        if (intval >> i)==0:
            bits.append(i)
            break
    else:
        raise ValueError('value {} doesnt seem to be valid'.format(intval))

bittable = table.Table(data=[bits, names, descrs], names=['bit', 'name', 'description'])
bittable = table.join(bittable, bitcounttable)

Now add some comparison statistics

In [30]:
bittable['frac_rem'] = bittable['count']/len(remwflags)
bittable['frac_all'] = np.array([all_flag_counts[bitnum] for bitnum in bittable['bit']])/allcount

In [31]:
bittable.sort('count')
bittable.reverse()
bittable.show_in_notebook(display_length=10)

bit,name,description,count,frac_rem,frac_all
29,BINNED1,Object was detected in 1x1 binned image,419,0.997619047619,0.978002466211
5,CHILD,Object is the product of an attempt to deblend a BLENDED object.,380,0.904761904762,0.322745195518
46,DEBLENDED_AT_EDGE,"""An object close enough to the edge of the frame normally not deblended, is deblended anyway. Only set for objects large enough to be EDGE in all fields/strips.""",380,0.904761904762,0.322769077803
18,INTERP,Object contains one or more pixels whose values were determined by interpolation.,363,0.864285714286,0.492042812651
37,STATIONARY,This object was consistent with being stationary.,341,0.811904761905,0.888019866162
9,NOPETRO,No valid Petrosian radius was found for this object.,302,0.719047619048,0.79654827877
26,DEBLENDED_AS_PSF,Deblender treated obj as PSF,274,0.652380952381,0.13368961803
47,DEBLEND_NOPEAK,There was no detected peak within this child in at least one band.,264,0.628571428571,0.132301521155
13,COSMIC_RAY,Contains a pixel interpreted to be part of a cosmic ray.,223,0.530952380952,0.174045317854
49,TOO_FEW_GOOD_DETECTIONS,A child of this object had too few good detections to be deblended as moving.,161,0.383333333333,0.464396874832
